In [70]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
from time import strftime
from time import sleep
import warnings
warnings.filterwarnings("ignore")
import json
import unicodedata
import re

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import acquire
from prepare import *

In [71]:
data = pd.read_json('data.json')

In [72]:
data = data.dropna()

In [73]:
data

,repo,language,readme_contents
1,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith..."
3,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
5,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...
7,ianstormtaylor/slate,TypeScript,"<p align=""center"">\n <a href=""#""><img src=""./..."
8,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...
...,...,...,...
555,tornadoweb/tornado,Python,Tornado Web Server\n==================\n\n.. i...
556,emscripten-core/emscripten,C,![emscripten logo](media/switch_logo.png)\n\nM...
557,gulpjs/gulp,JavaScript,"<p align=""center"">\n <a href=""https://gulpjs...."
558,alibaba/weex,C++,# Weex\n\nA framework for building Mobile cros...


In [110]:
data.readme_contents[0]

'<p>\n    <a href="https://tailwindcss.com/" target="_blank">\n      <img alt="Tailwind CSS" width="350" src="https://refactoringui.nyc3.cdn.digitaloceanspaces.com/tailwind-logo-sticker.svg">\n    </a><br>\n    A utility-first CSS framework for rapidly building custom user interfaces.\n</p>\n\n<p>\n    <a href="https://github.com/tailwindlabs/tailwindcss/actions"><img src="https://img.shields.io/github/workflow/status/tailwindlabs/tailwindcss/Node.js%20CI" alt="Build Status"></a>\n    <a href="https://www.npmjs.com/package/tailwindcss"><img src="https://img.shields.io/npm/dt/tailwindcss.svg" alt="Total Downloads"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/releases"><img src="https://img.shields.io/npm/v/tailwindcss.svg" alt="Latest Release"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/blob/master/LICENSE"><img src="https://img.shields.io/npm/l/tailwindcss.svg" alt="License"></a>\n</p>\n\n------\n\n## Documentation\n\nFor full documentation, visit [ta

In [104]:
data.readme_contents[2:3]

2    <p align="center">\n  <a href="#"><img src="./...
Name: readme_contents, dtype: object

In [76]:
data= data[data['language'].map(data['language'].value_counts()) >= 30]

In [77]:
data = data.reset_index()

In [95]:
response = response.to_string()

In [98]:
response

'0    <p>\\n    <a href="https://tailwindcss.com/" ta...'

In [97]:
soup = BeautifulSoup(response)

In [101]:
soup

<html><body><p>0    </p><p>\n    <a href="https://tailwindcss.com/" ta...=""></a></p></body></html>

In [84]:
response = data[0:1].readme_contents
soup = BeautifulSoup(response.text)

AttributeError: 'Series' object has no attribute 'text'

In [42]:
data = prep_readme_data(df= data, column = 'readme_contents', extra_words=['https','p','com','http','href'], exclude_words=[])

In [43]:
data.head()

,readme_contents,repo,language,clean,stemmed,lemmatized
0,"<p>\n <a href=""https://tailwindcss.com/"" ta...",tailwindlabs/tailwindcss,JavaScript,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...
1,# window.fetch polyfill\n\nThe `fetch()` funct...,github/fetch,JavaScript,windowfetch polyfill fetch function promisebas...,windowfetch polyfil fetch function promisebas ...,windowfetch polyfill fetch function promisebas...
2,"<p align=""center"">\n <a href=""#""><img src=""./...",ianstormtaylor/slate,TypeScript,aligncenter hrefimg srcdocsimagesbannerpng ali...,aligncent hrefimg srcdocsimagesbannerpng align...,aligncenter hrefimg srcdocsimagesbannerpng ali...
3,# Insomnia REST Client\n\n[![Slack Channel](ht...,Kong/insomnia,JavaScript,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...
4,![](https://user-images.githubusercontent.com/...,CymChad/BaseRecyclerViewAdapterHelper,Java,httpsuserimagesgithubusercontentcom76982093319...,httpsuserimagesgithubusercontentcom76982093319...,httpsuserimagesgithubusercontentcom76982093319...


In [44]:
data = data.drop(columns = ['readme_contents'])

In [50]:
data.language.value_counts()

JavaScript    141
Python         59
Java           46
TypeScript     45
Go             36
C++            30
Name: language, dtype: int64

In [51]:
js_df = data[data.language == 'JavaScript']
python_df = data[data.language == 'Python']
java_df = data[data.language == 'Java']
ts_df = data[data.language == 'TypeScript']
go_df = data[data.language == 'Go']
c_df = data[data.language == 'C++']

In [58]:
js_df.clean[0]

'hrefhttpstailwindcsscom target_blank img alttailwind css width350 srchttpsrefactoringuinyc3cdndigitaloceanspacescomtailwindlogostickersvg abr utilityfirst css framework rapidly building custom user interfaces hrefhttpsgithubcomtailwindlabstailwindcssactionsimg srchttpsimgshieldsiogithubworkflowstatustailwindlabstailwindcssnodejs20ci altbuild statusa hrefhttpswwwnpmjscompackagetailwindcssimg srchttpsimgshieldsionpmdttailwindcsssvg alttotal downloadsa hrefhttpsgithubcomtailwindcsstailwindcssreleasesimg srchttpsimgshieldsionpmvtailwindcsssvg altlatest releasea hrefhttpsgithubcomtailwindcsstailwindcssblobmasterlicenseimg srchttpsimgshieldsionpmltailwindcsssvg altlicensea documentation full documentation visit tailwindcsscomhttpstailwindcsscom community help discussion best practices conversation would benefit searchable discuss tailwind css githubhttpsgithubcomtailwindcsstailwindcssdiscussions casual chitchat others using framework join tailwind css discord serverhttpsdiscordgg7nf8gne con